In [1]:
import pandas as pd
import numpy as np 
import zipcodes

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pathlib
import sys
sys.path.append(str(pathlib.Path().absolute().parent))

In [2]:
#city_w_zip.columns

NameError: name 'city_w_zip' is not defined

In [ ]:
#zip_w_city.columns

In [9]:
# Load Data from data/interim directory:
zip_w_city = pd.read_pickle('../data/interim/ca-zip-w-city-ts.pickle')
city_w_zip = pd.read_pickle('../data/interim/zip_time_series.csv.pickle')

ts_data = city_w_zip

ts_data.tail()

,Date,RegionName,InventorySeasonallyAdjusted_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,MedianListingPricePerSqft_AllHomes,...,ZHVI_BottomTier,ZHVI_CondoCoop,ZHVI_MiddleTier,ZHVI_SingleFamilyResidence,ZHVI_TopTier,ZRI_AllHomes,ZRI_AllHomesPlusMultifamily,ZriPerSqft_AllHomes,Zri_MultiFamilyResidenceRental,Zri_SingleFamilyResidenceRental
4383880,2017-12-31,99709,57.0,34.0,NaN,NaN,NaN,NaN,NaN,176.886792,...,176200.0,135400.0,239600.0,242700.0,320400.0,NaN,NaN,NaN,NaN,NaN
4383881,2017-12-31,99712,43.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,...,178700.0,NaN,255900.0,255900.0,321300.0,NaN,NaN,NaN,NaN,NaN
4383882,2017-12-31,99801,80.0,74.0,NaN,NaN,228.168542,NaN,NaN,219.997535,...,268100.0,174700.0,342600.0,353400.0,456200.0,NaN,NaN,NaN,NaN,NaN
4383883,2017-12-31,99835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4383884,2017-12-31,99901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,186.440678,...,167500.0,122700.0,249500.0,267100.0,337500.0,NaN,NaN,NaN,NaN,NaN


In [10]:
#need to figure out the number of months between the last date we have and todays date. 
#in theory ts_date['Date'] would be for the zip code we need and not just the last zipcode we have in the table.
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month
date_today = datetime.today().strftime('%Y-%m-%d')
print(date_today)
latest_date = max(dt for dt in ts_data['Date'] if dt < date_today)
print(latest_date)
prediction_size = diff_month(datetime.strptime(date_today, "%Y-%m-%d"),datetime.strptime(latest_date, "%Y-%m-%d"))
print(prediction_size)

2020-03-30
2017-12-31
27


In [13]:
zip_ts = ts_data.loc[ts_data['RegionName'] == 91770]

my_data = zip_ts.rename(columns={'Date': 'ds','ZriPerSqft_AllHomes': 'y'})
my_data.tail()

,ds,RegionName,InventorySeasonallyAdjusted_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,MedianListingPricePerSqft_AllHomes,...,ZHVI_BottomTier,ZHVI_CondoCoop,ZHVI_MiddleTier,ZHVI_SingleFamilyResidence,ZHVI_TopTier,ZRI_AllHomes,ZRI_AllHomesPlusMultifamily,y,Zri_MultiFamilyResidenceRental,Zri_SingleFamilyResidenceRental
4305943,2017-08-31,91770,39.0,41.0,NaN,NaN,NaN,NaN,NaN,389.632782,...,487900.0,429100.0,550400.0,557100.0,690400.0,2440.0,2430.0,1.810,2362.0,2443.0
4324990,2017-09-30,91770,34.0,37.0,NaN,NaN,NaN,NaN,NaN,384.097035,...,494100.0,434500.0,554900.0,560800.0,697200.0,2436.0,2429.0,1.814,2361.0,2441.0
4344037,2017-10-31,91770,33.0,35.0,NaN,NaN,NaN,NaN,NaN,389.150943,...,499900.0,440700.0,558600.0,564300.0,705400.0,2433.0,2426.0,1.816,2368.0,2439.0
4363084,2017-11-30,91770,33.0,36.0,NaN,NaN,NaN,NaN,NaN,370.544160,...,504300.0,449000.0,563100.0,569700.0,718400.0,2436.0,2430.0,1.818,2374.0,2444.0
4382131,2017-12-31,91770,32.0,30.0,NaN,NaN,NaN,NaN,NaN,376.024377,...,507400.0,456100.0,569300.0,576500.0,729700.0,2442.0,2434.0,1.820,2380.0,2450.0


In [14]:
# Transformations necessary for FB Prophet
# the date column must be a datetime type and labeled 'ds'
# the column we wish teo forecast must be labeled 'y'

#Skye Note: It looks like you are only renaming the date column and not the y... 
# I think the y should be named in "user_specified_BR"


my_model = Prophet(interval_width=0.95)
my_model.fit(my_data)
future = my_model.make_future_dataframe(periods=prediction_size)
forecast = my_model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

#this is forcasting the price per sqft per all homes so we would just multiply this by the SQft they are looking at
#however, if they are looking at a sqft range I'm not sure how we will handle that. 
# we could do a min and max so we would have two different projections...

#price based on number of bedrooms would be easier.

#then we are taking this data and putting it into the math equation for the rent or buy

#but we need to figure out the prediction for renting a house in the same zip

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat,yhat_lower,yhat_upper
283,2018-01-23,1.815217,1.808758,1.821473
284,2018-01-24,1.817075,1.810717,1.823075
285,2018-01-25,1.819405,1.813218,1.826288
286,2018-01-26,1.822178,1.816100,1.828495
287,2018-01-27,1.825355,1.818982,1.832108


In [ ]:
#Looking at Renting a similar house in the same zip code...

In [15]:
forecast['yhat'].iloc[-1]
#set y
# run model
#set y, rental
#run prophet


1.8253546739360285